In [ ]:
%pip install fuzzywuzzy

In [7]:
import pandas as pd
from fuzzywuzzy import fuzz, process

# Cargar los archivos
bd_refacciones = pd.read_csv("/content/BD REFACCIONES.csv", encoding="utf-8")
bd_sap = pd.read_csv("/content/BD SAP.csv", encoding="utf-8")

# Normalizar texto
def normalizar_texto(texto):
    return str(texto).strip().lower()

bd_refacciones["Refaccion"] = bd_refacciones["Refaccion"].apply(normalizar_texto)
bd_sap["Descripcion"] = bd_sap["Descripcion"].apply(normalizar_texto)

# Función para buscar coincidencias
def buscar_coincidencia(refaccion, sap_data, threshold=60):
    if sap_data.empty:
        return None
    coincidencia = process.extractOne(refaccion, sap_data["Descripcion"], scorer=fuzz.partial_ratio)
    if coincidencia and coincidencia[1] >= threshold:
        # Validar si el índice obtenido es válido
        idx = coincidencia[2]
        if 0 <= idx < len(sap_data):
            return sap_data.iloc[idx]
    return None

# Proceso principal
resultado = []

for _, ref_row in bd_refacciones.iterrows():
    montacargas = ref_row["Montacargas"]
    refaccion = ref_row["Refaccion"]

    # Filtrar por montacargas
    sap_local = bd_sap[bd_sap["Montacargas"] == montacargas]
    match_local = None

    if not sap_local.empty:
        match_local = buscar_coincidencia(refaccion, sap_local)

    if match_local is not None:
        resultado.append({
            "Montacargas": montacargas,
            "Refaccion": refaccion,
            "SAP": match_local["SAP"],
            "Descripcion": match_local["Descripcion"],
            "Tipo de Coincidencia": "Alta" if fuzz.ratio(refaccion, match_local["Descripcion"]) >= 80 else "Media",
            "Ambito de Busqueda": "Local"
        })
    else:
        # Búsqueda global
        match_global = buscar_coincidencia(refaccion, bd_sap)
        if match_global is not None:
            resultado.append({
                "Montacargas": montacargas,
                "Refaccion": refaccion,
                "SAP": match_global["SAP"],
                "Descripcion": match_global["Descripcion"],
                "Tipo de Coincidencia": "Alta" if fuzz.ratio(refaccion, match_global["Descripcion"]) >= 80 else "Media",
                "Ambito de Busqueda": "Global"
            })
        else:
            # Sin coincidencia
            resultado.append({
                "Montacargas": montacargas,
                "Refaccion": refaccion,
                "SAP": "Sin coincidencia",
                "Descripcion": "Sin coincidencia",
                "Tipo de Coincidencia": "Sin coincidencia",
                "Ambito de Busqueda": "Sin coincidencia"
            })

# Convertir resultados a DataFrame
df_resultado = pd.DataFrame(resultado)

# Guardar en un archivo CSV
df_resultado.to_csv("Resultado_Coincidencias.csv", index=False, encoding="utf-8")

print("Proceso completado. Revisa el archivo 'Resultado_Coincidencias.csv'.")


Proceso completado. Revisa el archivo 'Resultado_Coincidencias.csv'.


In [1]:
pip install pytesseract pdf2image PyPDF2 pymupdf reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.6 MB/s eta 0:00:00


In [ ]:
# Importar las bibliotecas
from pdf2image import convert_from_path
import pytesseract
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from PyPDF2 import PdfWriter, PdfReader
import os

# Ruta al archivo PDF original
pdf_path = "archivo.pdf"  # Cambia a la ruta de tu archivo PDF

# Convertir PDF a imágenes
print("Convirtiendo el PDF a imágenes...")
pages = convert_from_path(pdf_path, dpi=300)
image_paths = []
for i, page in enumerate(pages):
    image_path = f"page_{i}.png"
    page.save(image_path, "PNG")
    image_paths.append(image_path)

# Realizar OCR en cada imagen y guardar los textos
print("Realizando OCR en las imágenes...")
ocr_texts = []
for image_path in image_paths:
    text = pytesseract.image_to_string(image_path, lang='spa')  # OCR en español
    ocr_texts.append(text)

# Crear un nuevo PDF con texto seleccionable
output_pdf_path = "pdf_con_texto.pdf"
print("Creando el PDF con texto seleccionable...")
writer = PdfWriter()
for i, (image_path, text) in enumerate(zip(image_paths, ocr_texts)):
    # Crear un PDF temporal con el texto OCR
    temp_pdf_path = f"temp_page_{i}.pdf"
    c = canvas.Canvas(temp_pdf_path, pagesize=letter)
    c.drawString(50, 750, text[:100])  # Ejemplo: texto al inicio de la página
    c.drawImage(image_path, 0, 0, width=letter[0], height=letter[1])  # Insertar imagen
    c.save()

    # Añadir la página al PDF final
    writer.append(temp_pdf_path)

# Escribir el PDF final
with open(output_pdf_path, "wb") as output_file:
    writer.write(output_file)

# Limpieza de archivos temporales
for image_path in image_paths:
    os.remove(image_path)
for i in range(len(ocr_texts)):
    os.remove(f"temp_page_{i}.pdf")

print(f"PDF con texto seleccionable creado: {output_pdf_path}")
